In [1]:
import os

In [2]:
%pwd

'c:\\Projects\\Text-Summarizer\\research'

In [3]:
os.chdir('../')

In [4]:
%pwd

'c:\\Projects\\Text-Summarizer'

In [5]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_URL: str
    local_data_file: Path
    unzip_dir: Path

In [6]:
pip install PyYAML


Note: you may need to restart the kernel to use updated packages.


In [7]:
import sys
import os

# Add 'src' folder to system path
sys.path.append(os.path.join(os.getcwd(), "src"))

# Now import
from TextSummarizer.constants import CONFIG_FILE_PATH
print(CONFIG_FILE_PATH)


config\config.yaml


In [13]:
from TextSummarizer.constants import *
from TextSummarizer.utils.common import read_yaml, create_directories

In [14]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])

    

    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion

        create_directories([config.root_dir])

        data_ingestion_config = DataIngestionConfig(
            root_dir=config.root_dir,
            source_URL=config.source_URL,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir 
        )

        return data_ingestion_config

In [15]:
import os
import urllib.request as request
import zipfile
from TextSummarizer.logging import logger
from TextSummarizer.utils.common import get_size

In [16]:
class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config


    
    def download_file(self):
        if not os.path.exists(self.config.local_data_file):
            filename, headers = request.urlretrieve(
                url = self.config.source_URL,
                filename = self.config.local_data_file
            )
            logger.info(f"{filename} download! with following info: \n{headers}")
        else:
            logger.info(f"File already exists of size: {get_size(Path(self.config.local_data_file))}")  

        
    
    def extract_zip_file(self):
        """
        zip_file_path: str
        Extracts the zip file into the data directory
        Function returns None
        """
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)

In [17]:
# print("YAML path exists?", CONFIG_FILE_PATH.exists())
# print("YAML path:", CONFIG_FILE_PATH)


In [18]:
# from pathlib import Path

# CONFIG_FILE_PATH = Path("config/config.yaml")


In [19]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise e

[2025-07-07 01:09:21,146: INFO: common: yaml file: config\config.yaml loaded successfully]
[2025-07-07 01:09:21,169: INFO: common: yaml file: params.yaml loaded successfully]
[2025-07-07 01:09:21,174: INFO: common: created directory at: artifacts]
[2025-07-07 01:09:21,177: INFO: common: created directory at: artifacts/data_ingestion]


[2025-07-07 01:09:26,393: INFO: 1434958058: artifacts/data_ingestion/data.zip download! with following info: 
Connection: close
Content-Length: 3825349
Cache-Control: max-age=300
Content-Security-Policy: default-src 'none'; style-src 'unsafe-inline'; sandbox
Content-Type: application/zip
ETag: "67a57a032da4fbc785772cfcf18db91ca12af11c022ed516ffefb42b064b2d04"
Strict-Transport-Security: max-age=31536000
X-Content-Type-Options: nosniff
X-Frame-Options: deny
X-XSS-Protection: 1; mode=block
X-GitHub-Request-Id: D86D:1D8549:22C52C:570BD1:686AD0EF
Accept-Ranges: bytes
Date: Sun, 06 Jul 2025 19:39:27 GMT
Via: 1.1 varnish
X-Served-By: cache-bom4749-BOM
X-Cache: MISS
X-Cache-Hits: 0
X-Timer: S1751830767.174020,VS0,VE552
Vary: Authorization,Accept-Encoding
Access-Control-Allow-Origin: *
Cross-Origin-Resource-Policy: cross-origin
X-Fastly-Request-ID: 0d6cea417b94d2655c03c59648f30ef3733fad9d
Expires: Sun, 06 Jul 2025 19:44:27 GMT
Source-Age: 0

]
